# Sorting & Ordering in Polars

### Loading Libraries

In [33]:
# ZipFiles & IO
import io
import zipfile

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import polars as pl
import polars.selectors as cs

# Data Visualization
import hvplot
import seaborn as sns
import altair as alt
import matplotlib.pyplot as plt

# XGBoost
import xgboost as xgb

# Scikit-Learn
from sklearn import decomposition

# Date & Time
from datetime import datetime
from datetime import timedelta

In [2]:
hvplot.extension('matplotlib')

## Loading Data

In [3]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [4]:
raw = pl.read_csv(path, null_values=['NA'])

In [5]:
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany')                    
                          .str.contains('Automatic')
                          .fill_null('Automatic'),
                          num_gears=pl.col('trany')
                          .str.extract(r'(\d+)')
                          .cast(pl.UInt8)
                          .fill_null(6))
           )

In [6]:
autos = tweak_auto(raw)

In [7]:
print(autos)

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

### Sorting by A Single Column

In [8]:
names = ['Al', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank']

In [9]:
sorted(names)

['Al', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank']

In [10]:
# Sorted by String's Length
sorted(names, key=len)

['Al', 'Bob', 'Dan', 'Edith', 'Frank', 'Charlie']

In [11]:
print(autos
      .with_columns(pl.col('make').cast(pl.String))
      .sort(by=pl.col('make').str.len_chars())
     )

shape: (48_231, 15)
┌──────┬─────────────┬─────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make        ┆ model       ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---         ┆ ---         ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ str         ┆ cat         ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆             ┆             ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪═════════════╪═════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1993 ┆ BMW         ┆ 740i        ┆ 4.0   ┆ … ┆ 20        ┆ 2013-01-01  ┆ true        ┆ 5         │
│      ┆             ┆             ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1993 ┆ BMW         ┆ 740il       ┆ 4.0   ┆ … ┆ 20        ┆ 2013-01-01  ┆ true        ┆ 5         │
│      ┆             ┆             ┆       ┆   ┆           ┆ 00:00:00  

In [12]:
print(autos
      .sort('year')
     )

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1984 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1984 ┆ Bertone    ┆ X1/9         ┆ 1.5   ┆ … ┆ 26        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [13]:
print(autos
      .sort((pl.col('city08') + pl.col('highway08')) / 2)
     )

shape: (48_231, 15)
┌──────┬───────┬────────────────┬───────┬───┬───────────┬───────────────┬──────────────┬───────────┐
│ year ┆ make  ┆ model          ┆ displ ┆ … ┆ highway08 ┆ createdOn     ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---   ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---           ┆ ---          ┆ ---       │
│ i16  ┆ cat   ┆ cat            ┆ f32   ┆   ┆ u8        ┆ datetime[μs]  ┆ str          ┆ u8        │
╞══════╪═══════╪════════════════╪═══════╪═══╪═══════════╪═══════════════╪══════════════╪═══════════╡
│ 2025 ┆ Lucid ┆ Air Pure RWD   ┆ null  ┆ … ┆ 125       ┆ 2024-08-25    ┆ true         ┆ 1         │
│      ┆       ┆ with 20 inch   ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│      ┆       ┆ whee…          ┆       ┆   ┆           ┆               ┆              ┆           │
│ 2024 ┆ Lucid ┆ Air G Touring  ┆ null  ┆ … ┆ 128       ┆ 2024-05-02    ┆ true         ┆ 1         │
│      ┆       ┆ XR AWD with19  ┆       ┆   ┆           ┆ 00:00:00     

In [14]:
# Sorting VClass
print(autos
      .sort(pl.col('VClass'))
     )

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [15]:
print(autos
      .with_columns(make_avg=pl.col('city08').mean().over(pl.col('make')))
     )

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬─────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ make_avg  │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ f64       │
│      ┆            ┆              ┆       ┆   ┆ ]           ┆ str         ┆           ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 19.5      │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 11.911972 │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00    ┆         

In [16]:
print(autos
      .sort(pl.col('city08').mean().over(pl.col('make')))
     )

shape: (48_231, 15)
┌──────┬─────────┬───────────────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make    ┆ model         ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---     ┆ ---           ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat     ┆ cat           ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪═════════╪═══════════════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
│ 1996 ┆ Vector  ┆ Avtech SC /   ┆ 5.7   ┆ … ┆ 12        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆         ┆ M12           ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1997 ┆ Vector  ┆ Avtech SC /   ┆ 5.7   ┆ … ┆ 12        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆         ┆ M12           ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1992 ┆ Vector  ┆ W8            ┆ 6.0   ┆ … ┆ 10        ┆ 2013-01-01  

### Sorting by Multiple Columns

In [17]:
print(autos
      .sort(['year', 'make'])
     )

shape: (48_231, 15)
┌──────┬─────────────┬─────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make        ┆ model       ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---         ┆ ---         ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat         ┆ cat         ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆             ┆             ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪═════════════╪═════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1984 ┆ Alfa Romeo  ┆ GT V6 2.5   ┆ 2.5   ┆ … ┆ 24        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆             ┆             ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1984 ┆ Alfa Romeo  ┆ Spider      ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆             ┆ Veloce 2000 ┆       ┆   ┆           ┆ 00:00:00  

In [18]:
# Including Model
print(autos
      .sort(['year', 'make', 'model'])
     )

shape: (48_231, 15)
┌──────┬─────────────┬─────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make        ┆ model       ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---         ┆ ---         ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat         ┆ cat         ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆             ┆             ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪═════════════╪═════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1984 ┆ Alfa Romeo  ┆ Spider      ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆             ┆ Veloce 2000 ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1984 ┆ Alfa Romeo  ┆ Spider      ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆             ┆ Veloce 2000 ┆       ┆   ┆           ┆ 00:00:00  

In [19]:
# Descending Order
print(autos
      .sort(['year', 'make', 'model'],
           descending=[True, False, False])
     )

shape: (48_231, 15)
┌──────┬─────────────┬─────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make        ┆ model       ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---         ┆ ---         ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat         ┆ cat         ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆             ┆             ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪═════════════╪═════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 2025 ┆ Alfa Romeo  ┆ Giulia      ┆ 2.0   ┆ … ┆ 33        ┆ 2024-08-07  ┆ true        ┆ 8         │
│      ┆             ┆             ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 2025 ┆ Alfa Romeo  ┆ Giulia AWD  ┆ 2.0   ┆ … ┆ 31        ┆ 2024-08-07  ┆ true        ┆ 8         │
│      ┆             ┆             ┆       ┆   ┆           ┆ 00:00:00  

In [20]:
print(autos
      .with_columns(pl.col('make').cast(pl.Categorical('lexical')),
                    pl.col('model').cast(pl.Categorical('lexical')))
      .sort(['year', 'make', 'model'],
            descending=[True, False, False])
     )

shape: (48_231, 15)
┌──────┬───────┬─────────┬───────┬───┬───────────┬──────────────────────┬──────────────┬───────────┐
│ year ┆ make  ┆ model   ┆ displ ┆ … ┆ highway08 ┆ createdOn            ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---   ┆ ---     ┆ ---   ┆   ┆ ---       ┆ ---                  ┆ ---          ┆ ---       │
│ i16  ┆ cat   ┆ cat     ┆ f32   ┆   ┆ u8        ┆ datetime[μs]         ┆ str          ┆ u8        │
╞══════╪═══════╪═════════╪═══════╪═══╪═══════════╪══════════════════════╪══════════════╪═══════════╡
│ 2025 ┆ Acura ┆ Integra ┆ 1.5   ┆ … ┆ 37        ┆ 2024-05-29 00:00:00  ┆ true         ┆ 7         │
│ 2025 ┆ Acura ┆ Integra ┆ 2.0   ┆ … ┆ 28        ┆ 2024-08-25 00:00:00  ┆ false        ┆ 6         │
│ 2025 ┆ Acura ┆ Integra ┆ 1.5   ┆ … ┆ 36        ┆ 2024-05-29 00:00:00  ┆ false        ┆ 6         │
│      ┆       ┆ A-Spec  ┆       ┆   ┆           ┆                      ┆              ┆           │
│ 2025 ┆ Acura ┆ Integra ┆ 1.5   ┆ … ┆ 36        ┆ 2024-05-29 00:00:00 

### Specifying Custom Ordering for Categorical Columns

In [22]:
print(autos
      .with_columns(month=pl.col('createdOn').dt.strftime('%B'))
      .sort('month')
     )

shape: (48_231, 16)
┌──────┬───────────┬──────────────┬───────┬───┬──────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make      ┆ model        ┆ displ ┆ … ┆ createdOn    ┆ is_automati ┆ num_gears ┆ month     │
│ ---  ┆ ---       ┆ ---          ┆ ---   ┆   ┆ ---          ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat       ┆ cat          ┆ f32   ┆   ┆ datetime[μs] ┆ ---         ┆ u8        ┆ str       │
│      ┆           ┆              ┆       ┆   ┆              ┆ str         ┆           ┆           │
╞══════╪═══════════╪══════════════╪═══════╪═══╪══════════════╪═════════════╪═══════════╪═══════════╡
│ 2014 ┆ Porsche   ┆ Panamera S   ┆ 3.0   ┆ … ┆ 2014-04-02   ┆ true        ┆ 8         ┆ April     │
│      ┆           ┆ E-Hybrid     ┆       ┆   ┆ 00:00:00     ┆             ┆           ┆           │
│ 2014 ┆ MINI      ┆ Cooper       ┆ 1.5   ┆ … ┆ 2014-04-02   ┆ true        ┆ 6         ┆ April     │
│      ┆           ┆ (3-doors)    ┆       ┆   ┆ 00:00:00     ┆         

In [23]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 
               'December']

In [24]:
with pl.StringCache():
    pl.Series(month_order).cast(pl.Categorical)
    print(autos
        .with_columns(month=pl.col('createdOn').dt.strftime('%B')
                      .cast(pl.Categorical))
        .sort('month')
         )

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬──────────────┬─────────────┬───────────┬──────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn    ┆ is_automati ┆ num_gears ┆ month    │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---          ┆ c           ┆ ---       ┆ ---      │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs] ┆ ---         ┆ u8        ┆ cat      │
│      ┆            ┆              ┆       ┆   ┆              ┆ str         ┆           ┆          │
╞══════╪════════════╪══════════════╪═══════╪═══╪══════════════╪═════════════╪═══════════╪══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01   ┆ false       ┆ 5         ┆ January  │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00     ┆             ┆           ┆          │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01   ┆ false       ┆ 5         ┆ January  │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00     ┆        

In [25]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 
               'December']

In [26]:
# An Alternative
@pl.StringCache()
def create_month_order():
    s = pl.Series(month_order).cast(pl.Categorical)
    return(autos
        .with_columns(month=pl.col('createdOn').dt.strftime('%B')
                      .cast(pl.Categorical))
          )

In [27]:
print(create_month_order().sort('month'))

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬──────────────┬─────────────┬───────────┬──────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn    ┆ is_automati ┆ num_gears ┆ month    │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---          ┆ c           ┆ ---       ┆ ---      │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs] ┆ ---         ┆ u8        ┆ cat      │
│      ┆            ┆              ┆       ┆   ┆              ┆ str         ┆           ┆          │
╞══════╪════════════╪══════════════╪═══════╪═══╪══════════════╪═════════════╪═══════════╪══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01   ┆ false       ┆ 5         ┆ January  │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00     ┆             ┆           ┆          │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01   ┆ false       ┆ 5         ┆ January  │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00     ┆        

#### Applying Decorator on `tweak_auto` Function()

In [28]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [29]:
raw = pl.read_csv(path, null_values=['NA'])

In [30]:
@pl.StringCache()
... def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany',
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08',
            'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col('make').cast(pl.Categorical('lexical')),
                          pl.col(['model', 'VClass', 'drive', 'fuelType'])
            .cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany').str.contains('Auto'),
                          num_gears=pl.col('trany').str.extract(r'(\d+)').cast(pl.Int8)
                         )
           )

In [31]:
autos = tweak_auto(raw)

In [32]:
print(tweak_auto(raw)
      .sort('make'))

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ i8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ bool        ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ AM General ┆ Post Office  ┆ 2.5   ┆ … ┆ 17        ┆ 2013-01-01  ┆ true        ┆ 3         │
│      ┆            ┆ DJ5 2WD      ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ AM General ┆ Post Office  ┆ 4.2   ┆ … ┆ 13        ┆ 2013-01-01  ┆ true        ┆ 3         │
│      ┆            ┆ DJ8 2WD      ┆       ┆   ┆           ┆ 00:00:00  

### Enums & Ordering

##### Placing an Example Birthday's Dataset

In [34]:
data = '''Name, Birthday
    Brianna Smith,2000-02-16
    Alex Johnson,2001-01-15
    Carlos Gomez,2002-03-17
    Diana Clarke,2003-04-18
    Ethan Hunt,2002-05-19
    Fiona Gray,2005-06-20
    George King,2006-07-21
    Hannah Scott,2007-08-22
    Ian Miles,2008-09-23
    Julia Banks,2009-10-24'''

In [35]:
# IO Reading
students = pl.read_csv(io.StringIO(data))

In [36]:
print(students)

shape: (10, 2)
┌───────────────────┬────────────┐
│ Name              ┆ Birthday   │
│ ---               ┆ ---        │
│ str               ┆ str        │
╞═══════════════════╪════════════╡
│     Brianna Smith ┆ 2000-02-16 │
│     Alex Johnson  ┆ 2001-01-15 │
│     Carlos Gomez  ┆ 2002-03-17 │
│     Diana Clarke  ┆ 2003-04-18 │
│     Ethan Hunt    ┆ 2002-05-19 │
│     Fiona Gray    ┆ 2005-06-20 │
│     George King   ┆ 2006-07-21 │
│     Hannah Scott  ┆ 2007-08-22 │
│     Ian Miles     ┆ 2008-09-23 │
│     Julia Banks   ┆ 2009-10-24 │
└───────────────────┴────────────┘


In [37]:
bday = pl.col('Birthday')

In [38]:
print(students
      .with_columns(bday.str.to_datetime('%Y-%m-%d'))
      .with_columns(month=bday.dt.strftime('%B'))
      .sort('month')
     ) 

shape: (10, 3)
┌───────────────────┬─────────────────────┬───────────┐
│ Name              ┆ Birthday            ┆ month     │
│ ---               ┆ ---                 ┆ ---       │
│ str               ┆ datetime[μs]        ┆ str       │
╞═══════════════════╪═════════════════════╪═══════════╡
│     Diana Clarke  ┆ 2003-04-18 00:00:00 ┆ April     │
│     Hannah Scott  ┆ 2007-08-22 00:00:00 ┆ August    │
│     Brianna Smith ┆ 2000-02-16 00:00:00 ┆ February  │
│     Alex Johnson  ┆ 2001-01-15 00:00:00 ┆ January   │
│     George King   ┆ 2006-07-21 00:00:00 ┆ July      │
│     Fiona Gray    ┆ 2005-06-20 00:00:00 ┆ June      │
│     Carlos Gomez  ┆ 2002-03-17 00:00:00 ┆ March     │
│     Ethan Hunt    ┆ 2002-05-19 00:00:00 ┆ May       │
│     Julia Banks   ┆ 2009-10-24 00:00:00 ┆ October   │
│     Ian Miles     ┆ 2008-09-23 00:00:00 ┆ September │
└───────────────────┴─────────────────────┴───────────┘


In [39]:
# Placing for Chronological Order
month_type = pl.Enum(['January', 'February', 'March', 'April', 'May', 
                      'June', 'July', 'August', 'September', 'October', 'November', 
                      'December'])

In [40]:
print(students
      .with_columns(bday.str.to_datetime('%Y-%m-%d'))
      .with_columns(month=bday.dt.strftime('%B').cast(month_type))
      .sort('month')
     ) 

shape: (10, 3)
┌───────────────────┬─────────────────────┬───────────┐
│ Name              ┆ Birthday            ┆ month     │
│ ---               ┆ ---                 ┆ ---       │
│ str               ┆ datetime[μs]        ┆ enum      │
╞═══════════════════╪═════════════════════╪═══════════╡
│     Alex Johnson  ┆ 2001-01-15 00:00:00 ┆ January   │
│     Brianna Smith ┆ 2000-02-16 00:00:00 ┆ February  │
│     Carlos Gomez  ┆ 2002-03-17 00:00:00 ┆ March     │
│     Diana Clarke  ┆ 2003-04-18 00:00:00 ┆ April     │
│     Ethan Hunt    ┆ 2002-05-19 00:00:00 ┆ May       │
│     Fiona Gray    ┆ 2005-06-20 00:00:00 ┆ June      │
│     George King   ┆ 2006-07-21 00:00:00 ┆ July      │
│     Hannah Scott  ┆ 2007-08-22 00:00:00 ┆ August    │
│     Ian Miles     ┆ 2008-09-23 00:00:00 ┆ September │
│     Julia Banks   ┆ 2009-10-24 00:00:00 ┆ October   │
└───────────────────┴─────────────────────┴───────────┘


##### Merging on Holidays Dataset

In [41]:
holiday_data = '''Month,Holiday
    January,New Year's Day
    February,Valentine's Day
    March,St. Patrick's Day
    April,April Fools' Day
    May,Memorial Day
    June,Juneteenth
    July,Independence Day
    August,Labor Day
    September,Patriot Day
    October,Halloween
    November,Thanksgiving
    December,Christmas Day'''

In [42]:
holidays = pl.read_csv(io.StringIO(holiday_data))

In [43]:
print(holidays)

shape: (12, 2)
┌───────────────┬───────────────────┐
│ Month         ┆ Holiday           │
│ ---           ┆ ---               │
│ str           ┆ str               │
╞═══════════════╪═══════════════════╡
│     January   ┆ New Year's Day    │
│     February  ┆ Valentine's Day   │
│     March     ┆ St. Patrick's Day │
│     April     ┆ April Fools' Day  │
│     May       ┆ Memorial Day      │
│ …             ┆ …                 │
│     August    ┆ Labor Day         │
│     September ┆ Patriot Day       │
│     October   ┆ Halloween         │
│     November  ┆ Thanksgiving      │
│     December  ┆ Christmas Day     │
└───────────────┴───────────────────┘


In [44]:
print(students
      .with_columns(bday.str.to_datetime('%Y-%m-%d'))
      .with_columns(month=bday.dt.strftime('%B'))
      .join(holidays, left_on='month', right_on='Month')
      .sort('month'))

shape: (0, 4)
┌──────┬──────────────┬───────┬─────────┐
│ Name ┆ Birthday     ┆ month ┆ Holiday │
│ ---  ┆ ---          ┆ ---   ┆ ---     │
│ str  ┆ datetime[μs] ┆ str   ┆ str     │
╞══════╪══════════════╪═══════╪═════════╡
└──────┴──────────────┴───────┴─────────┘


In [45]:
# print(students
#       .with_columns(bday.str.to_datetime('%Y-%m-%d'))
#       .with_columns(month=bday.dt.strftime('%B').cast(pl.Categorical))
#       .join(holidays, left_on='month', right_on='Month')
#       .sort('month'))

In [46]:
print(students
      .with_columns(bday.str.to_datetime('%Y-%m-%d'))
      .with_columns(month=bday.dt.strftime('%B').cast(pl.Categorical))
      .join(holidays.with_columns(pl.col('Month').cast(pl.Categorical)),
            left_on='month', right_on='Month')
      .sort('month'))

shape: (0, 4)
┌──────┬──────────────┬───────┬─────────┐
│ Name ┆ Birthday     ┆ month ┆ Holiday │
│ ---  ┆ ---          ┆ ---   ┆ ---     │
│ str  ┆ datetime[μs] ┆ cat   ┆ str     │
╞══════╪══════════════╪═══════╪═════════╡
└──────┴──────────────┴───────┴─────────┘


/var/folders/dk/f9pbhbp52qxc0613mpbtx8lm0000gn/T/ipykernel_12630/3260817562.py:4: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .join(holidays.with_columns(pl.col('Month').cast(pl.Categorical)),


In [48]:
# print(students
#       .with_columns(bday.str.to_datetime('%Y-%m-%d'))
#       .with_columns(month=bday.dt.strftime('%B').cast(month_type))
#       .join(holidays.with_columns(pl.col('Month').cast(month_type)),
#             left_on='month', right_on='Month')
#       .sort('month'))

### Group Ordering & `maintain_order`

In [49]:
print(autos
      .group_by(pl.col('make'))
      .len()
     )

shape: (145, 2)
┌─────────────────────────────┬──────┐
│ make                        ┆ len  │
│ ---                         ┆ ---  │
│ cat                         ┆ u32  │
╞═════════════════════════════╪══════╡
│ Lexus                       ┆ 699  │
│ Maserati                    ┆ 214  │
│ Chevrolet                   ┆ 4472 │
│ Jaguar                      ┆ 540  │
│ Pontiac                     ┆ 893  │
│ …                           ┆ …    │
│ Autokraft Limited           ┆ 4    │
│ Panther Car Company Limited ┆ 4    │
│ Bentley                     ┆ 177  │
│ Ruf Automobile Gmbh         ┆ 3    │
│ Bugatti Rimac               ┆ 2    │
└─────────────────────────────┴──────┘


In [50]:
print(autos
      .group_by(pl.col('make'), maintain_order=True)
      .len()
     )

shape: (145, 2)
┌──────────────────────────┬──────┐
│ make                     ┆ len  │
│ ---                      ┆ ---  │
│ cat                      ┆ u32  │
╞══════════════════════════╪══════╡
│ Alfa Romeo               ┆ 98   │
│ Ferrari                  ┆ 284  │
│ Dodge                    ┆ 2696 │
│ Subaru                   ┆ 1013 │
│ Toyota                   ┆ 2470 │
│ …                        ┆ …    │
│ General Motors           ┆ 1    │
│ Consulier Industries Inc ┆ 3    │
│ Goldacre                 ┆ 1    │
│ Isis Imports Ltd         ┆ 1    │
│ PAS Inc - GMC            ┆ 2    │
└──────────────────────────┴──────┘


In [51]:
print(autos
      .group_by(pl.col('make'))
      .len()
      .sort('make')
     )

shape: (145, 2)
┌─────────────────────────────┬──────┐
│ make                        ┆ len  │
│ ---                         ┆ ---  │
│ cat                         ┆ u32  │
╞═════════════════════════════╪══════╡
│ AM General                  ┆ 6    │
│ ASC Incorporated            ┆ 1    │
│ Acura                       ┆ 424  │
│ Alfa Romeo                  ┆ 98   │
│ American Motors Corporation ┆ 27   │
│ …                           ┆ …    │
│ Volkswagen                  ┆ 1326 │
│ Volvo                       ┆ 933  │
│ Wallace Environmental       ┆ 32   │
│ Yugo                        ┆ 8    │
│ smart                       ┆ 38   │
└─────────────────────────────┴──────┘


### Stable Sorting

In [52]:
students = pl.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Dana', 'Eve'],
    'age': [25, 20, 25, 21, 24],
    'grade': [88, 92, 95, 88, 60],
})

In [53]:
print(students)

shape: (5, 3)
┌─────────┬─────┬───────┐
│ name    ┆ age ┆ grade │
│ ---     ┆ --- ┆ ---   │
│ str     ┆ i64 ┆ i64   │
╞═════════╪═════╪═══════╡
│ Alice   ┆ 25  ┆ 88    │
│ Bob     ┆ 20  ┆ 92    │
│ Charlie ┆ 25  ┆ 95    │
│ Dana    ┆ 21  ┆ 88    │
│ Eve     ┆ 24  ┆ 60    │
└─────────┴─────┴───────┘


In [54]:
print(students
      .sort('age')
     )

shape: (5, 3)
┌─────────┬─────┬───────┐
│ name    ┆ age ┆ grade │
│ ---     ┆ --- ┆ ---   │
│ str     ┆ i64 ┆ i64   │
╞═════════╪═════╪═══════╡
│ Bob     ┆ 20  ┆ 92    │
│ Dana    ┆ 21  ┆ 88    │
│ Eve     ┆ 24  ┆ 60    │
│ Alice   ┆ 25  ┆ 88    │
│ Charlie ┆ 25  ┆ 95    │
└─────────┴─────┴───────┘


In [55]:
print(students
      .sort('grade')
     )

shape: (5, 3)
┌─────────┬─────┬───────┐
│ name    ┆ age ┆ grade │
│ ---     ┆ --- ┆ ---   │
│ str     ┆ i64 ┆ i64   │
╞═════════╪═════╪═══════╡
│ Eve     ┆ 24  ┆ 60    │
│ Alice   ┆ 25  ┆ 88    │
│ Dana    ┆ 21  ┆ 88    │
│ Bob     ┆ 20  ┆ 92    │
│ Charlie ┆ 25  ┆ 95    │
└─────────┴─────┴───────┘


### Sorting & Filtering

In [57]:
%%timeit
(autos.filter(pl.col('year') == 1994))

242 μs ± 1.22 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [58]:
autos_year_sorted = autos.sort('year')

In [59]:
%%timeit
(autos_year_sorted.filter(pl.col('year') == 1994))

65.6 μs ± 1.22 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [60]:
cat_make = autos

In [61]:
string_make = autos.with_columns(make=pl.col('make').cast(pl.String))

In [62]:
sorted_make = (autos
               .with_columns(make=pl.col('make').cast(pl.String))
               .sort('make'))

In [63]:
%%timeit
cat_make.filter(pl.col('make') == 'Ford')

546 μs ± 1.39 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [64]:
%%timeit
string_make.filter(pl.col('make') == 'Ford')

109 μs ± 577 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [65]:
%%timeit
sorted_make.filter(pl.col('make') == 'Ford')

95.6 μs ± 920 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
